In [6]:
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
from scipy import stats
from scipy.stats import norm
from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pypfopt import EfficientFrontier, risk_models, expected_returns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
from IPython.display import display, clear_output

ticker_text = widgets.Text(
    value='AAPL, MSFT, TSLA',
    description='Tickers:',
    placeholder='Enter tickers separated by commas'
)
display(ticker_text)

inputs_box = widgets.VBox()
display(inputs_box)

create_inputs_button = widgets.Button(description="Create Inputs")
display(create_inputs_button)

# Run analysis button
run_button = widgets.Button(description="Run Analysis")
display(run_button)

output = widgets.Output()
display(output)

quantity_inputs = {}  # To hold slider widgets keyed by ticker

def on_create_inputs_clicked(b):
    with output:
        clear_output()
        # Parse tickers
        tickers = [t.strip().upper() for t in ticker_text.value.split(',') if t.strip()]
        if not tickers:
            print("Please enter at least one ticker.")
            return
        # Clear old inputs
        inputs_box.children = []
        quantity_inputs.clear()

        # Create input field for each ticker
        for t in tickers:
            input_box = widgets.IntText(value=1, description=t, min=0)
            quantity_inputs[t] = input_box
        inputs_box.children = list(quantity_inputs.values())

# Hook up the button
create_inputs_button.on_click(on_create_inputs_clicked)

def on_run_clicked(b):
    with output:
        clear_output()
        # parse tickers from text input
        tickers = [t.strip().upper() for t in ticker_text.value.split(',') if t.strip()]
        if not tickers:
            print("No tickers found. Please enter tickers and create sliders.")
            return
        
        # check quantity_inputs sliders exist for these tickers
        missing = [t for t in tickers if t not in quantity_inputs]
        if missing:
            print(f"Please click 'Create Sliders' again to generate sliders for: {missing}")
            return

        units = np.array([quantity_inputs[t].value for t in tickers])

        end = dt.datetime.now()
        start = dt.datetime(2023,10,1)

        df = yf.download(['^GSPC'] + tickers, start, end, auto_adjust=True)
        Close = df['Close'][['^GSPC'] + tickers]

        log_returns = np.log(Close / Close.shift(1)).dropna()

        def calc_beta(df):
            np_array = df.values
            m = np_array[:, 0]
            beta = []
            for ind, col in enumerate(df):
                if ind > 0:
                    s = np_array[:, ind]
                    covariance = np.cov(s, m)
                    if covariance[1,1] == 0:
                        beta.append(np.nan)
                    else:
                        beta.append(covariance[0,1] / covariance[1,1])
            return pd.Series(beta, df.columns[1:], name='Beta')

        beta = calc_beta(log_returns)
        stocklist = yf.download(tickers, start, end, auto_adjust=True)
        sprices = stocklist['Close'].iloc[-1][tickers].values
        price = np.array([round(p,2) for p in sprices])
        value = units * price
        weight = [round(val/sum(value),2) for val in value]
        beta = round(beta, 2)

        portfolio = pd.DataFrame({
            'Stock': tickers,
            'Price': np.ravel(price),
            'Units': np.ravel(units),
            'Current Value': np.ravel(value),
            'Weights': np.ravel(weight),
            'Beta': np.ravel(beta),
            'Weighted Beta': np.ravel(weight) * np.ravel(beta)
        })

        portfolio.set_index('Stock', inplace=True)

        index = yf.download(['^GSPC'], start, end, multi_level_index=False)
        SP500price = index.Close.iloc[-1]

        portfolio['SP500 Weighted Delta (point)'] = round(portfolio['Beta'] * portfolio['Price'] / SP500price * portfolio['Units'], 2)
        portfolio['SP500 Weighted Delta (1%)'] = round(portfolio['Beta'] * portfolio['Price'] * portfolio['Units'] * 0.01, 2)

        portfolio.loc['Total', ['Current Value', 'SP500 Weighted Delta (point)', 'SP500 Weighted Delta (1%)']] = \
        portfolio[['Current Value', 'SP500 Weighted Delta (point)', 'SP500 Weighted Delta (1%)']].sum()

        display(portfolio)
        
        sp500 = yf.download('^GSPC', start, end, multi_level_index = False)
        
        # Extract Close prices from stocklist (already downloaded)
        close_prices = stocklist['Close'][tickers]
        
        # Create units dataframe for broadcasting
        units_df = pd.DataFrame(units, index=tickers).T
        
        # Calculate portfolio daily value series
        portfolio_ts = (close_prices * units_df.values).sum(axis=1)

        fig = go.Figure()
        fig = make_subplots(specs=[[{"secondary_y": True}]])


        fig.add_trace(go.Scatter(
            x=portfolio_ts.index,
            y=portfolio_ts.values,
            mode='lines',
            name='Portfolio Value'
        ), secondary_y = False)

        fig.add_trace(go.Scatter(
            x=sp500.index,
            y=sp500['Close'],
            mode='lines',
            name='SP 500'
        ), secondary_y = True)

        fig.update_yaxes(title_text="Portfolio Value", secondary_y=False)
        fig.update_yaxes(title_text="SP500", secondary_y=True)
        
        fig.update_layout(
            title='Portfolio Timeline',
            xaxis_title='Date',
            yaxis_title='Total Value',
            template='plotly_white',
            height=600,
            width=1000
        )
        
        fig.show()

        log_tfsa_returns = np.log(portfolio_ts/portfolio_ts.shift(1)).dropna()
        conf_lvl = 0.95
        VaR = np.percentile(log_tfsa_returns, (1 - conf_lvl) * 100) 
        CVaR = log_tfsa_returns[log_tfsa_returns <= VaR].mean() 

        VaR_pct = VaR * 100
        CVaR_pct = CVaR * 100

        fig2data = portfolio_ts.pct_change().dropna()

        fig2 = go.Figure(data=[go.Histogram(x=fig2data * 100, nbinsx=500)])

        fig2.add_vline(x=VaR_pct, line=dict(color="red", width=2, dash="dash"), name='VaR')
        fig2.add_vline(x=CVaR_pct, line=dict(color="darkred", width=2, dash="dot"), name='CVaR')

        fig2.add_annotation(x=VaR_pct, y=5, text=f"VaR 95%: {VaR_pct:.2f}%", showarrow=True, arrowhead=1, ax=20)
        fig2.add_annotation(x=CVaR_pct, y=5, text=f"CVaR 95%: {CVaR_pct:.2f}%", showarrow=True, arrowhead=1, ax=-80)

        fig2.update_layout(
            title='Portfolio VaR, CVaR, and daily returns',
            xaxis_title='Percent Change',
            yaxis_title='Frequency',
            template='plotly_white',
            height=600,
            width=1000
        )
        
        fig2.show()

        daily_std = log_tfsa_returns.std()
        annualized_vol = daily_std * np.sqrt(252)
        trading_days = 60 ##this is a trading window
        volatility = log_tfsa_returns.rolling(window=trading_days).std()*np.sqrt(trading_days) 

        sp500_log_returns = np.log(Close['^GSPC'] / Close['^GSPC'].shift(1)).dropna()
        total_return = np.exp(sp500_log_returns.sum()) - 1
        num_years = (Close.index[-1] - Close.index[0]).days / 365
        annual_sp500_return = (1 + total_return)**(1/num_years) - 1

        Rf = annual_sp500_return/252 ##risk free rate is sp500. 252 trading days
        sharpe_ratio = (log_tfsa_returns.rolling(window=trading_days).mean() - Rf)*trading_days/volatility

        fig3 = go.Figure()

        fig3.add_trace(go.Scatter(
            x=sharpe_ratio.index,
            y=sharpe_ratio.values,
            mode='lines',
            name='Sharpe Ratio TFSA'
        ))

        fig3.update_layout(
            title='Portfolio Sharpe Ratio',
            xaxis_title='Date',
            yaxis_title='Sharpe Ratio',
            template='plotly_white',
            height=600,
            width=1000
        )
        
        fig3.show()
        
        tickers = [t for t in portfolio.index if t.upper() != "TOTAL"]  
        units = portfolio.loc[tickers, 'Units'].values
        prices = portfolio.loc[tickers, 'Price'].values
        values = units * prices
        total_value = values.sum()

        current_weights = pd.Series(values / total_value, index=tickers)

        valid_tickers = tickers  

        current_weights = current_weights.reindex(valid_tickers).fillna(0)

        hist_prices2 = yf.download(valid_tickers, start, end)

        hist_prices = hist_prices2.Close

        mu = expected_returns.mean_historical_return(hist_prices)
        S = risk_models.sample_cov(hist_prices)

        ef = EfficientFrontier(mu, S)
        optimal_weights = ef.max_sharpe()
        cleaned_weights = ef.clean_weights()

        comparison_df = pd.DataFrame({
            'Current Weight': current_weights,
            'Optimal Weight': pd.Series(cleaned_weights).reindex(valid_tickers).fillna(0)
        }).sort_index()
        
        fig5 = go.Figure()

        fig5.add_trace(go.Bar(
            x=comparison_df.index,
            y=comparison_df['Current Weight'],
            name='Current Weight',
            marker_color='indianred'
        ))

        fig5.add_trace(go.Bar(
            x=comparison_df.index,
            y=comparison_df['Optimal Weight'],
            name='Optimal Weight',
            marker_color='lightskyblue'
        ))

        fig5.update_layout(
            title='Current vs Optimal Portfolio Weights',
            xaxis_title='Stock Ticker',
            yaxis_title='Weight',
            barmode='group',
            template='plotly_white',
            width=900,
            height=500
        )
        
        fig5.show()

        u = log_tfsa_returns.mean()
        v = log_tfsa_returns.var()
        drift = u - 0.5*v
        np.array(drift)
        norm.ppf(0.95)
        x = np.random.rand(10, 2)
        norm.ppf(x)
        Z = norm.ppf(np.random.rand(10, 2))
        t_intervals = 1000
        iterations = 250
        daily_returns = np.exp(drift + daily_std * norm.ppf(np.random.rand(t_intervals, iterations)))
        s0 = portfolio_ts.iloc[-1]
        price_list = np.zeros_like(daily_returns)
        price_list[0] = s0
        for t in range(1, t_intervals):
            price_list[t] = price_list[t-1] * daily_returns[t]

        fig4 = go.Figure()

        # Plot each simulation path
        for i in range(price_list.shape[1]):
            fig4.add_trace(go.Scatter(
                x=np.arange(t_intervals),
                y=price_list[:, i],
                mode='lines',
                name=f'Simulation {i+1}',
                line=dict(width=1),
                showlegend=False  # Optional to keep the plot clean
            ))

        # Calculate and plot the average (expected) path
        mean_path = price_list.mean(axis=1)
        fig4.add_trace(go.Scatter(
            x=np.arange(t_intervals),
            y=mean_path,
            mode='lines',
            name='Average Path',
            line=dict(color='black', width=3, dash='dash')  # dashed bold black line
        ))

        fig4.update_layout(
            title='Monte Carlo Simulation of Portfolio',
            xaxis_title='Time (Days)',
            yaxis_title='Portfolio Value',
            template='plotly_white',
            width=1000,
            height=600
        )

        fig4.show()

        monthly_prices = stocklist.Close.resample('ME').last()

        monthly_returns = monthly_prices.pct_change().dropna()

        weights_series = portfolio['Weights'].drop('Total', errors='ignore')
        tickers_monthly = monthly_returns.columns.tolist()
        weights_aligned = np.array([weights_series.get(ticker, 0) for ticker in tickers_monthly])
        portfolio_monthly_returns = monthly_returns.dot(weights_aligned)

        def create_sequences(data, seq_length=12):
            X, y = [], []
            for i in range(len(data) - seq_length):
                X.append(data[i:i+seq_length])
                y.append(data[i+seq_length])
            return np.array(X), np.array(y)

        seq_length = 12
        returns_values = portfolio_monthly_returns.values
        X, y = create_sequences(returns_values, seq_length)

        # 5) Scaling
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()
        X_scaled = scaler_X.fit_transform(X.reshape(-1, 1)).reshape(X.shape)
        y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

        # 6) Train/test split
        split = int(len(X_scaled) * 0.8)
        X_train, X_val = X_scaled[:split], X_scaled[split:]
        y_train, y_val = y_scaled[:split], y_scaled[split:]
        model = Sequential([
            Input(shape=(seq_length, 1)),
            LSTM(64, return_sequences=True),
            Dropout(0.2),
            LSTM(32),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')

        # 8) Train with early stopping
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                            epochs=100, batch_size=4, callbacks=[early_stop], verbose=1)

        # 10) Predict next month
        last_seq = returns_values[-seq_length:]
        last_seq_scaled = scaler_X.transform(last_seq.reshape(-1, 1)).reshape(1, seq_length, 1)
        pred_scaled = model.predict(last_seq_scaled)
        predicted_return = scaler_y.inverse_transform(pred_scaled)

        print(f"Predicted portfolio return next month: {predicted_return[0][0]:.4%}")


run_button.on_click(on_run_clicked)

Text(value='AAPL, MSFT, TSLA', description='Tickers:', placeholder='Enter tickers separated by commas')

VBox()

Button(description='Create Inputs', style=ButtonStyle())

Button(description='Run Analysis', style=ButtonStyle())

Output()

NameError: name 'on_create_inputs_clicked' is not defined